In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load data

In [79]:
import numpy as np
from utils.datasetHeads import MeshHeadsDataset

## Create dataset
 All the meshes come from the same deformed subject, so the points are different but the faces are the same.

In [80]:
images_folder = '/home/fmatzkin/Code/datasets/cq500/converted/selected/preprocessed_ct_to_skull/reg_CQ500-CT-12_CT Thin Plain_decimated_1perc/'
train_heads = open("train_heads.txt", 'r').read().splitlines()
dataset = MeshHeadsDataset(train_heads, images_folder)
print(f"Amount of meshes: {len(dataset)}")

Amount of meshes: 128


## Show example

In [81]:
import vedo

points = dataset[0]['image'].points()
vedo.Mesh(points).show().close()

# PCA
## Flatten

In [82]:
print(f"shape of the meshes: {dataset[0]['image'].points().shape}")

shape of the meshes: (6739, 3)


In [83]:
flat_meshes = []

for idx in range(len(dataset)):
    # print('\r', idx + 1, 'of', len(dataset), end='')
    mesh = dataset[idx]['image'].points()
    flat_mesh = mesh.reshape(1, -1)
    flat_meshes.append(flat_mesh)

flat_meshes = np.concatenate(flat_meshes, axis = 0)
print(f"flattened and concatenated {len(flat_meshes)} meshes")

flattened and concatenated 128 meshes


## Fit PCA

In [84]:
from sklearn.decomposition import PCA

pca = PCA(n_components=32)
pca.fit(flat_meshes[:64])

PCA(n_components=32)

## Predict with unseen images

In [114]:
faces = points = dataset[0]['image'].faces()
reconstr = pca.inverse_transform(pca.transform(flat_meshes[64:]))
for i, f_mesh in enumerate(reconstr):
    points = f_mesh.reshape(-1, 3)
    restored_mesh = vedo.Mesh([points, faces])
    restored_mesh.write(f'test/{i}.stl')
    print(f'\r saved mesh {i}/{len(reconstr)}', end='')


 saved mesh 63/64

## Save components

In [115]:
components = pca.components_
np.save("models/heads_pca_components.npy", components)

mean = pca.mean_
np.save("models/heads_pca_mean.npy", mean)